# ML.Net - Time-series forecasting

**Rational**:
Predict upcoming sales for a specific product

In [1]:
// ML.NET Nuget packages 
#r "nuget:Microsoft.ML"  
#r "nuget:Microsoft.ML.FastTree"
#r "nuget:Microsoft.ML.TimeSeries"
    
//Install XPlot package
#r "nuget:XPlot.Plotly"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML.TimeSeries version 1.5.0

Installed package XPlot.Plotly version 3.0.1

Installed package Microsoft.ML version 1.5.0

Installed package Microsoft.ML.FastTree version 1.5.0

In [2]:
using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;
using Microsoft.ML.Transforms.TimeSeries;

In [3]:
public class ProductData
{
    //Next month's #units sold
    [LoadColumn(0)]
    public float next;

    [LoadColumn(1)]
    public float productId;

    [LoadColumn(2)]
    public float year;

    [LoadColumn(3)]
    public float month;

    [LoadColumn(4)]
    public float units;

    //Average #units sold per day in a month
    [LoadColumn(5)]
    public float avg;

    [LoadColumn(6)]
    public float count;

    //Max #units sold per day in a month
    [LoadColumn(7)]
    public float max;

    //Min #units sold per day in a month
    [LoadColumn(8)]
    public float min;

    //Previous month's #units sold
    [LoadColumn(9)]
    public float prev;
}

In [4]:
public class ProductUnitTimeSeriesPrediction
{
    public float[] ForecastedProductUnits { get; set; }

    public float[] ConfidenceLowerBound { get; set; }

    public float[] ConfidenceUpperBound { get; set; }
}

In [5]:
var dataPath = "./Datasets/Time-Series Forecasting/products.stats.csv";

In [6]:
var mlContext = new MLContext(seed: 1);

var dataView = mlContext.Data.LoadFromTextFile<ProductData>(dataPath, hasHeader: true, separatorChar:',');

In [7]:
//Get data for a specific product
var productId = 988;

IDataView productDataView = mlContext.Data.FilterRowsByColumn(dataView, "productId", productId, productId + 1);

#### Data exploration

In [8]:
var rows = mlContext.Data.CreateEnumerable<ProductData>(productDataView, reuseRowObject: false)
                    .Take(5)
                    .ToList();

display(rows);

index,next,productId,year,month,units,avg,count,max,min,prev
0,971,988,2017,1,759,42,18,150,1,427
1,1175,988,2017,2,971,48,20,205,5,759
2,547,988,2017,3,1175,55,21,210,2,971
3,968,988,2017,4,547,28,19,108,6,1175
4,834,988,2017,5,968,40,24,161,5,547


#### Plotting

In [9]:
var years = productDataView.GetColumn<float>("year").ToArray();
var months = productDataView.GetColumn<float>("month").ToArray();

var index = 0;
var dates = new List<DateTime>();
foreach(var year in years)
{
    dates.Add(new DateTime(Convert.ToInt32(year), Convert.ToInt32(months[index]), 1));
    index++;
}

In [10]:
var units = productDataView.GetColumn<float>("units").ToArray();
var max = productDataView.GetColumn<float>("max").ToArray();
var min = productDataView.GetColumn<float>("min").ToArray();
var avg = productDataView.GetColumn<float>("avg").ToArray();

In [11]:
var chart = Chart.Plot(
    new Graph.Scatter()
    {
        x = dates,
        y = units,
        name = "Number of units"         
    },
    new Layout.Layout 
    { 
        title= "Units sold per month",
        yaxis = new Graph.Yaxis { rangemode = "tozero" }
    } 
);

chart.WithLegend(true);
display(chart);